In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import os
import sys
import pickle

from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

sys.path.append('../..')

device = "cuda"

data_home = "../../artefacts/data"
model_home = "../../artefacts/models/challenge_1"

SFREQ = 100

In [3]:
class Validator:
    def __init__(self, valid_loader, target):
        self.valid_loader = valid_loader
        self.target = torch.as_tensor(target, dtype=torch.float32)

    @staticmethod
    def z_to_predict(z, temperature=0.1):
        """softargmax предсказание на torch"""
        T = z.shape[-1]
        dt = 1 / 100
        win_offset = 0.5
        p = F.softmax(z / temperature, dim=-1)
        t_grid = torch.arange(T, device=z.device, dtype=z.dtype)[None, :] * dt
        t_hat_rel = (p * t_grid).sum(dim=-1)
        t_hat_abs = (t_hat_rel + win_offset).view(-1)
        return t_hat_abs

    @staticmethod
    def entropy_from_logits(z, temperature=1.0, eps=1e-12):
        """Энтропия распределений softmax по времени"""
        p = F.softmax(z / temperature, dim=-1)
        ent = -(p * (p.clamp_min(eps)).log()).sum(dim=-1)
        return ent

    @torch.no_grad()
    def validate_model(self, model, device='cuda', temperature=1.0):
        logits = []
        preds = []
        model.eval()
        for batch in tqdm(self.valid_loader):
            x = batch[0][:, :128, :].to(device).float()
            z = model(x).squeeze(1)  # (B,T)
            logits.append(z.cpu())
            preds.append(self.z_to_predict(z, temperature).cpu())
        logits = torch.cat(logits, dim=0)
        preds = torch.cat(preds, dim=0)
        _, nrmse, entropy = self.validate_logits(logits, temperature)
        return logits, preds, nrmse, entropy

    def validate_logits(self, logits, temperature):
        preds = self.z_to_predict(logits, temperature)
        entropy = self.entropy_from_logits(logits, temperature)
        rmse = torch.sqrt(((preds - self.target.to(preds.device)) ** 2).mean())
        nrmse = rmse / self.target.std()
        return preds, nrmse, entropy

## Loading the validation dataset

We not trained anything on our relatively big validation dataset (20% of subjects)

So now finally time to take advantage from this!

In [7]:
with open(os.path.join(data_home, "valid_dataset.pkl"), "rb") as f:
    valid_dataset = pickle.load(f)

meta_information_valid = valid_dataset.get_metadata()

## Searching the best folds split to have similar mean values across all folds

Should be reproducible

In [9]:
import numpy as np
from tqdm import tqdm
import itertools

# Compute per-subject mean target
subject_means = meta_information_valid.groupby('subject').target.mean()
subjects = subject_means.index.values
targets = subject_means.values
n_subjects = len(subjects)
n_folds = 5

# Create uniformly distributed initial fold assignments
base_fold_assignments = np.arange(n_subjects) % n_folds

# Generate permutations and search for the permutation with lowest std of fold means
best_std = float('inf')
best_permutation = None
best_subject_to_fold = None

# Approach: for 1000 iterations, permute fold assignments (uniform weights) and compute std
rng = np.random.default_rng(seed=42)
progress_bar = tqdm(range(10000), desc='Searching permutations for lowest std')
for _ in progress_bar:
    permuted_assignments = np.copy(base_fold_assignments)
    rng.shuffle(permuted_assignments)
    subject_to_fold = dict(zip(subjects, permuted_assignments))
    fold_assignments = meta_information_valid['subject'].map(subject_to_fold).values
    mean_per_fold = meta_information_valid.assign(fold=fold_assignments).groupby('fold').target.mean()
    fold_std = mean_per_fold.std()
    if fold_std < best_std:
        best_std = fold_std
        best_permutation = permuted_assignments.copy()
        best_subject_to_fold = subject_to_fold.copy()
        progress_bar.set_postfix(best_std=best_std)

# Assign best fold split to dataframe
meta_information_valid['fold'] = meta_information_valid['subject'].map(best_subject_to_fold).values
print(f"Best permutation found | std of fold target means: {best_std:.6f}")
display(meta_information_valid.groupby('fold').target.mean())
# meta_information_valid.to_csv('meta_valid_with_folds.csv', index=False)



Searching permutations for lowest std: 100%|██████████| 10000/10000 [01:12<00:00, 138.65it/s, best_std=0.00242]

Best permutation found | std of fold target means: 0.002420


fold
0    1.592597
1    1.592596
2     1.58827
3    1.589053
4    1.587539
Name: target, dtype: Float64

In [11]:
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=4)
validator = Validator(valid_loader, meta_information_valid['target'].values)

In [13]:
from neurosned.models.segmentation.factorization_unet import FactorizationSneddyUnet
from neurosned.models.segmentation.sneddy_unet import SneddySegUNet1D
from neurosned.models.segmentation.inception import EEGInceptionSeg1D
from neurosned.models.segmentation.attention_sneddy_unet import AttentionSneddyUnet

# unet_deeper_widen4_v0 - 903.4
# unet_deeper_widen4_v1 - 902.532
model_1 = SneddySegUNet1D(
    n_chans=128, n_times=200, sfreq=100,
    c0=64, widen=4, depth_per_stage=5, dropout=0.05, k=15,
    out_channels=1
).to(device)
model_1_weights = [
    "unet_deeper_widen4_v1"
]

# # unet_deeper_v0 - 903.320
# # unet_deeper_v1 - 901.810
# # unet_deeper_v3 - 899.418
# # unet_deeper_v4 - 898.900
# # unet_deeper_v5 - 898.618
model_2 = SneddySegUNet1D(
    n_chans=128, n_times=200, sfreq=100,
    c0=96, widen=2, depth_per_stage=5, dropout=0.05, k=15,
    out_channels=1
).to(device)
model_2_weights = [
    'unet_deeper_v4', 
]

model_3 = SneddySegUNet1D(
    n_chans=128, n_times=200, sfreq=100,
    c0=96, widen=2, depth_per_stage=3, dropout=0.05, k=15,
    out_channels=1
).to(device)
model_3_weights = ["unet_v4"] 

# attention_unet_v1 - 902.554
# attention_unet_v2 - 901.512
# attention_unet_v4 - 901.503
model_4 = AttentionSneddyUnet(
    n_chans=128, n_times=200, sfreq=100,
    c0=64, num_stages=5, widen=2.0, depth_per_stage=[2,2,2,1,1],
    bottleneck_type="mhsa", bottleneck_depth=3,
    attn_heads=4, attn_dropout=0.05, ffn_dropout=0.05,
    drop_path=0.1, skip_gating=True
).to(device)
model_4_weights = [
    'attention_unet_v2',
]

# Inception-style model
model_5 = EEGInceptionSeg1D(
    n_chans=128, n_times=200, sfreq=100,
    branch_out=32,
    scales_samples_s=(0.5, 0.25, 0.125),
    pooling_sizes=(1, 1),
    dropout=0.12,
    out_channels=1
).to(device)
model_5_weights = [
    "inception_v0"
]

# Factorization U-Net style model
model_6 = FactorizationSneddyUnet(
    n_chans=128, n_times=200, sfreq=100, c0=96, widen=2,
    n_stages=4,
    depth_per_stage=2, dropout=0.1, k=7, out_channels=1,
    fm_factors_front=64, fm_dropout_front=0.05,
    use_stage_fm=True,
    fm_factors_stage=32, fm_dropout_stage=0.05
).to(device)
model_6_weights = [
    "factorization_unet_v1_finetune",
]

model_weights_store = (
    (model_1, model_1_weights),
    (model_2, model_2_weights),
    (model_3, model_3_weights),
    (model_4, model_4_weights),
    (model_5, model_5_weights),
    (model_6, model_6_weights)
)

logits_store = {}
# entropy_store = {}
nrmse_store = {}
for model, model_fnames in model_weights_store:
    for fname in model_fnames:
        fpath = os.path.join(model_home, fname) + '.pth'
        current_state_dict = torch.load(fpath)
        model.load_state_dict(current_state_dict)
        logits, preds, nrmse, entropy = validator.validate_model(model)
        logits_store[fname] = logits
        nrmse_store[fname] = nrmse
        print(f"NRMSE: {nrmse:.6f} for {fname}")

100%|██████████| 181/181 [00:04<00:00, 36.40it/s]


NRMSE: 0.902532 for unet_deeper_widen4_v1


100%|██████████| 181/181 [00:04<00:00, 40.05it/s]


NRMSE: 0.898900 for unet_deeper_v4


100%|██████████| 181/181 [00:04<00:00, 39.50it/s]


...

NRMSE: 0.901511 for attention_unet_v2


100%|██████████| 181/181 [00:04<00:00, 40.58it/s]


NRMSE: 0.917440 for inception_v0


100%|██████████| 181/181 [00:09<00:00, 19.37it/s]

NRMSE: 0.910516 for factorization_unet_v1_finetune


In [14]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

def evaluate_models_nrmse(logits_store, targets, device=None, dt=1/100, win_offset=0.5):
    """
    Считает дефолтный и откалиброванный (по температуре) NRMSE для каждой модели в logits_store.

    logits_store: dict[str, np.ndarray или torch.Tensor], формы (N, T)
    targets: array-like длиной N (в секундах)
    device: 'cuda' или 'cpu'
    """
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    y = torch.as_tensor(targets, dtype=torch.float32, device=device)
    results = {}

    for name, logits in tqdm(logits_store.items(), desc="Evaluating models"):
        z = torch.as_tensor(logits, dtype=torch.float32, device=device)
        N, T = z.shape
        t_grid = torch.arange(T, device=device, dtype=torch.float32)[None, :] * dt

        # === 1. дефолтная температура (1.0) ===
        p = F.softmax(z / 1.0, dim=-1)
        t_hat = (p * t_grid).sum(dim=-1) + win_offset
        rmse = torch.sqrt(((t_hat - y) ** 2).mean())
        nrmse_def = (rmse / y.std()).item()

        # === 2. подбор температуры (грид-серч) ===
        best_T, best_nrmse = 1.0, nrmse_def
        for T in torch.linspace(0.4, 1.8, 30, device=device):  # сетка температур
            pT = F.softmax(z / T, dim=-1)
            t_hat_T = (pT * t_grid).sum(dim=-1) + win_offset
            rmse_T = torch.sqrt(((t_hat_T - y) ** 2).mean())
            nrmse_T = (rmse_T / y.std()).item()
            if nrmse_T < best_nrmse:
                best_nrmse, best_T = nrmse_T, float(T)

        results[name] = {
            "default_nrmse": nrmse_def,
            "calibrated_nrmse": best_nrmse,
            "best_temperature": best_T
        }

    # красивый вывод
    print(f"\n{'Model':30s} {'NRMSE_def':>12s} {'NRMSE_cal':>12s} {'Best_T':>8s}")
    print("-" * 70)
    for name, vals in results.items():
        print(f"{name:30s} {vals['default_nrmse']:.6f} {vals['calibrated_nrmse']:.6f} {vals['best_temperature']:.3f}")

    return results


In [15]:
targets = torch.as_tensor(meta_information_valid['target'].values, dtype=torch.float32)
results = evaluate_models_nrmse(logits_store, targets, device='cuda')

Evaluating models: 100%|██████████| 6/6 [00:00<00:00, 141.73it/s]


Model                             NRMSE_def    NRMSE_cal   Best_T
----------------------------------------------------------------------
unet_deeper_widen4_v1          0.902532 0.902324 1.076
unet_deeper_v4                 0.898900 0.898757 1.076
unet_v4                        0.903785 0.903785 1.000
attention_unet_v2              0.901511 0.901511 1.028
inception_v0                   0.917440 0.916910 1.124
factorization_unet_v1_finetune 0.910516 0.909281 1.221


# Stacking Preparation

In [18]:
from neurosned.wrappers.challenge_1.meta_regressor import RidgeMetaRegressor
from neurosned.wrappers.challenge_1.meta_features import MetaFeatureExtractor

fx_ridge = MetaFeatureExtractor(sfreq=100, win_offset=0.5, temps=(0.6, 1.0, 0.8))
X_train = fx_ridge.build_from_logits_store(logits_store, cls_outputs_store=None)

reg = RidgeMetaRegressor()     
folds = meta_information_valid.fold.values
reg.fit(X_train, meta_information_valid['target'].values, folds=folds)
reg.save("../../artefacts/models/challenge_1/meta_ridge_new.pkl")

Fold 0: RMSE 0.340538 | NRMSE 0.839932 | n=4649
Fold 1: RMSE 0.358408 | NRMSE 0.884007 | n=4467
Fold 2: RMSE 0.365506 | NRMSE 0.901516 | n=4590
Fold 3: RMSE 0.383917 | NRMSE 0.946925 | n=4674
Fold 4: RMSE 0.362830 | NRMSE 0.894915 | n=4669
OOF  : RMSE 0.362555 | NRMSE 0.894237 | n=23049


In [31]:
from neurosned.wrappers.challenge_1.meta_wrapper import MetaWrapper
from neurosned.wrappers.challenge_1.meta_regressor import MetaRegressor

seg_models = [model_1, model_2, model_3, model_4, model_5, model_6]  # ваши torch-модели
reg = MetaRegressor.load("../../artefacts/models/challenge_1/meta_ridge_new.pkl")

meta_ridge = MetaWrapper(seg_models=seg_models, cls_models=[],
                   feature_extractor=fx_ridge, meta_regressor=reg,
                   use_channels=np.arange(128), device=device).to(device)

# Boosting

In [19]:
import re
import numpy as np

def make_monotonic_constraints(feature_names, time_dir=1):
    """
    time_dir: 1 для времени-«вперёд», -1 если таргет убывает с реальным временем.
    """
    cst = []
    for n in feature_names:
        is_time = (
            n.endswith("t_hard")
            or ("t_abs_temp" in n)
            or bool(re.search(r"q\d+_temp", n))
        )
        cst.append(time_dir if is_time else 0)
    return np.asarray(cst, dtype=int)

In [20]:
from neurosned.wrappers.challenge_1.meta_regressor import HgbMetaRegressor
from neurosned.wrappers.challenge_1.meta_features import MetaFeatureExtractor

fx_hgbr = MetaFeatureExtractor(sfreq=100, win_offset=0.5, temps=(0.5, 0.7, 0.8, 1.))
X_train, colnames = fx_hgbr.build_from_logits_store(logits_store, cls_outputs_store=None, return_names=True)
cst = make_monotonic_constraints(colnames, time_dir=1)
print(cst)
print(colnames)
defaults_hgbr = {
    "loss": "squared_error",
    "quantile": None,
    "learning_rate": 0.01,
    "max_iter": 2000,
    "max_leaf_nodes": 20,
    "max_depth": 4,
    "min_samples_leaf": 30,
    "l2_regularization": 1.0,
    "max_features": 0.1,
    "max_bins": 100,
    "interaction_cst": 'pairwise',
    "early_stopping": "auto",
    "monotonic_cst": cst,
    "scoring": "loss",
    "validation_fraction": None,
    "n_iter_no_change": 50,
    "tol": 1e-7,
    "verbose": 0,
    "random_state": 42,
}
reg = HgbMetaRegressor(hgb_params=defaults_hgbr)         
folds = meta_information_valid.fold.values
reg.fit(X_train, meta_information_valid['target'].values, folds=folds)
reg.save("../../artefacts/models/challenge_1/meta_hgb_new.pkl")

[1 0 0 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0
 0 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 1
 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 1 0 0
 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0
 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1
 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1]
['seg_unet_deeper_widen4_v1_t_hard', 'seg_unet_deeper_widen4_v1_z_max', 'seg_unet_deeper_widen4_v1_z_margin', 'seg_unet_deeper_widen4_v1_t_abs_temp0.5', 'seg_unet_deeper_widen4_v1_ent_temp0.5', 'seg_unet_deeper_widen4_v1_pmax_temp0.5', 'seg_unet_deeper_widen4_v1_pmargin_temp0.5', 'seg_unet_deeper_widen4_v1_tvar_temp0.5', 'seg_unet_deeper_widen4_v1_q10_temp0.5', 'seg_unet_deeper_widen4_v1_q50_temp0.5', 'seg_unet_deeper_widen4_v1_q90_temp0.5', 'seg_unet_deeper_widen4_v1_t_abs_temp0.7', 'seg_unet_deeper_widen4_v1_ent_temp0.7', 'seg_unet_deeper_widen4_v1_pmax_temp0.7', '

## Online evaluation

In [28]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

# --- Metrics ---
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=np.float32)
    y_pred = np.asarray(y_pred, dtype=np.float32)
    return float(np.sqrt(np.mean((y_pred - y_true) ** 2)))

def nrmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=np.float32)
    return rmse(y_true, y_pred) / float(np.std(y_true))

# --- Stratified split by target ---
def stratified_split_by_target(y, n_splits=5, seed=42):
    y = np.asarray(y, dtype=np.float32)
    qs = np.quantile(y, np.linspace(0, 1, 11))
    y_bins = np.digitize(y, qs[1:-1])
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    return next(skf.split(np.zeros_like(y_bins), y_bins))

# --- Inference of a wrapper on a loader ---
@torch.no_grad()
def predict_with_wrapper(wrapper, loader, device="cuda"):
    wrapper.eval()
    preds = []
    for batch in tqdm(loader, desc="Infer", leave=False):
        if isinstance(batch, (list, tuple)):
            x = batch[0]
        else:
            x = batch
        x = x.to(device).float()
        y_hat = wrapper(x).squeeze(1)  # (B,)
        preds.append(y_hat.detach().cpu().numpy())
    return np.concatenate(preds, axis=0)

# --- Main evaluation function ---
def evaluate_submit_wrapper(wrapper, dataset, targets,
                            batch_size=256, num_workers=4,
                            n_splits=5, seed=42, device=None):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    wrapper.to(device)

    # train/test indices
    train_idx, test_idx = stratified_split_by_target(targets, n_splits=n_splits, seed=seed)

    # subsets and loaders
    train_ds = Subset(dataset, train_idx)
    test_ds  = Subset(dataset, test_idx)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=num_workers)

    # predictions
    y_pred_train = predict_with_wrapper(wrapper, train_loader, device=device)
    y_pred_test  = predict_with_wrapper(wrapper, test_loader,  device=device)

    # targets by indices
    y_train = np.asarray(targets)[train_idx]
    y_test  = np.asarray(targets)[test_idx]

    # metrics
    rmse_tr, nrmse_tr = rmse(y_train, y_pred_train), nrmse(y_train, y_pred_train)
    rmse_te, nrmse_te = rmse(y_test,  y_pred_test),  nrmse(y_test,  y_pred_test)

    print(f"Train: RMSE={rmse_tr:.6f}, NRMSE={nrmse_tr:.6f} | size={len(train_idx)}")
    print(f"Test : RMSE={rmse_te:.6f}, NRMSE={nrmse_te:.6f} | size={len(test_idx)}")

    return {
        "train_idx": train_idx,
        "test_idx": test_idx,
        "y_pred_train": y_pred_train,
        "y_pred_test": y_pred_test,
        "rmse_train": rmse_tr,
        "nrmse_train": nrmse_tr,
        "rmse_test": rmse_te,
        "nrmse_test": nrmse_te,
    }


In [ ]:
from neurosned.wrappers.challenge_1.meta_wrapper import MetaWrapper
from neurosned.wrappers.challenge_1.meta_regressor import  MetaRegressor

seg_models = [model_1, model_2, model_3, model_4, model_5, model_6]
reg = MetaRegressor.load("../../artefacts/models/challenge_1/meta_hgb_new.pkl")
fx_hgbr = MetaFeatureExtractor(sfreq=100, win_offset=0.5, temps=(0.5, 0.7, 0.8, 1.))
meta_hgbr = MetaWrapper(seg_models=seg_models, cls_models=[],
                   feature_extractor=fx_hgbr, meta_regressor=reg,
                   use_channels=np.arange(128), device=device).to(device)

## Gradient Boosting Performance as submission wrapper

In [ ]:
report = evaluate_submit_wrapper(
    wrapper=meta_hgbr,
    dataset=valid_dataset,         
    targets=meta_information_valid['target'].values,  
    batch_size=256,
    num_workers=4,
    n_splits=5,
    seed=42,
    device=device,
)

Train: RMSE=0.358030, NRMSE=0.880704 | size=18439
Test : RMSE=0.354263, NRMSE=0.883365 | size=4610


## Manual Blending performance as submission wrapper

In [ ]:
from pkg.wrappers.submit_wrapper import SubmitWrapper

wrapper = SubmitWrapper(
        segmentation_models=[
                            model_1, model_2, model_3, model_4, 
        ], 
        seg_weights=[       0.3,    0.35,    0.15,    0.2,   ],
        use_channels=np.arange(128), temperature=0.92
    )


report = evaluate_submit_wrapper(
    wrapper=wrapper,
    dataset=valid_dataset,         
    targets=meta_information_valid['target'].values,  
    batch_size=256,
    num_workers=4,
    n_splits=5,
    seed=42,
    device=device,
)

Train: RMSE=0.360137, NRMSE=0.889574 | size=18439
Test : RMSE=0.367009, NRMSE=0.899979 | size=4610


## Ridge Stacking performance as submission wrapper

In [ ]:
report = evaluate_submit_wrapper(
    wrapper=meta_ridge,
    dataset=valid_dataset,          
    targets=meta_information_valid['target'].values, 
    batch_size=256,
    num_workers=4,
    n_splits=5,
    seed=42,
    device=device,
)

Train: RMSE=0.360112, NRMSE=0.885826 | size=18439
Test : RMSE=0.357096, NRMSE=0.890430 | size=4610
